## 03 - Obtener la Matriz $A_{jk}$

Se obtendrá a continuación la matriz definida mediante esta expresión matemática para cada uno de los *boreholes* y se guardará en un ".csv".

$A_{jk} = erfc\left\lbrace \frac{z_j}{2 \sqrt{kt_{k-1}}}\right\rbrace - erfc\left\lbrace \frac{z_j}{2 \sqrt{kt_{k}}}\right\rbrace
$

In [2]:
# Procesamiento de datos
import numpy as np
import pandas as pd

# Paquetes para realizar gráficos
#import matplotlib.pyplot as plt
import seaborn as sns

# Paquetes importantes para el problema que vamos a realizar
import math
import os
import warnings
warnings.filterwarnings("ignore")

# Especifico el estilo que quiero en mis gráficos
sns.set_theme(style="whitegrid")

In [3]:
# Leer los datos de el datafrmae
df = pd.read_csv("../boreholes/dataframe.csv",delimiter=",",decimal=".",index_col="Unnamed: 0")

In [4]:
#Obtengo en una lista donde comenzarían los datos en cada caso
posicion = df["Posición del Primer Dato"]
names = df.index

#Path de donde tengo los datos 
document_name = []
for filename in os.listdir("../boreholes/raw"):
    document_name.append(os.path.join("../boreholes/raw", filename))

In [5]:
# Funciones últiles
#=============================================================
# Definir la matriz Ajk
def matriz(tk:list,zj:list,alpha:float):
    #Defino y lleno la matriz de 0-s
    Ajk = []
    filas = len(zj)
    columnas = len(tk)
    
    for i in range(filas):
        Ajk.append([])
        for j in range(columnas):
            Ajk[i].append(0)

    # La relleno con los datos
    for j in range(filas):
        for k in range(columnas):
            Ajk[j][k] = math.erfc(zj[j] / (2 * math.sqrt(alpha * tk[k-1])))
    
    return Ajk

#========================================================================
# Número de datos en profundidades mayores a 40m
def menor(zj:list,depth:int)->int:
    suma = 0 
    for i in zj:
        if(i<depth):
            suma += 1
    return suma

In [7]:
# Defino varios parámetros
tk = list(range(20,520,20))
alpha = 31.6 #m**2*y**-1

In [13]:
# Cálculo de la matriz para cada uno de los boreholes
menor_datos=[]
for i in range(len(names)):
    zj,Tj = np.loadtxt (document_name[i],skiprows = posicion[i]+1 ,unpack = True,encoding="latin1")
    Ajk = matriz(tk,zj,alpha)

    #Elimino una columna
    Ail = np.delete(Ajk,0,1)

    # Elimino x filas, x siendo el número de datos que tenemos en los 
    # primeros 40m
    Ail = Ail [menor(zj,40):]
    suma = menor(zj,40)

    menor_datos.append(int(suma))

    # Obtener un datafrmae para guardar los datos
    df_matriz = pd.DataFrame(Ail)
    df_matriz.to_csv("../resultados/matriz/matriz_{}.csv".format(names[i]),index = False)



In [17]:
# Guardo el número de filas extraídas para cada uno de los boreholes
np.savetxt("../resultados/datos/menor_40m.csv",menor_datos,delimiter=",",fmt="%s")